In [1]:
!wget https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q2_K.gguf

--2025-04-03 02:02:12--  https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q2_K.gguf
Resolving huggingface.co (huggingface.co)... 18.172.134.124, 18.172.134.24, 18.172.134.88, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.124|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/b0/ca/b0cae82fd4b3a362cab01d17953c45edac67d1c2dfb9fbb9e69c80c32dc2012e/c0dd304d761e8e05d082cc2902d7624a7f87858fdfaa4ef098330ffe767ff0d3?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27llama-2-7b-chat.Q2_K.gguf%3B+filename%3D%22llama-2-7b-chat.Q2_K.gguf%22%3B&Expires=1743649332&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MzY0OTMzMn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy9iMC9jYS9iMGNhZTgyZmQ0YjNhMzYyY2FiMDFkMTc5NTNjNDVlZGFjNjdkMWMyZGZiOWZiYjllNjljODBjMzJkYzIwMTJlL2MwZGQzMDRkNzYxZThlMDVkMDgyY2MyOTAyZDc2MjRhN2Y4Nzg1OGZkZmFhNGVmMDk4MzMwZmZlN

In [2]:
!pip install python-dotenv langchain faiss-cpu huggingface_hub transformers llama-cpp-python pypdf jsonpath-ng

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 MB 9.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.8-cp311-cp311-linux_x86_64.whl size=5959614 sha256=73966440ca7448d3a128efa96ce4e0ac391f88b52926db22faa42c2287af7036
  Stored in directory: /root/.cache/pip/wheels/c0/03/66/eb3810eafd55d921b2be32896d1f44313996982360663aa80b
Successfully built llama-cpp-python


In [4]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.7 MB/s eta 0:00:00


https://huggingface.co/spaces/namkwonwoo/hw9_rag_ai_chatbot/blob/main/app.py

شغال جيد

In [1]:
from dotenv import load_dotenv
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings  # General embeddings from HuggingFace models.
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import LlamaCpp  # For loading transformer models.
from langchain.document_loaders import PyPDFLoader, TextLoader, JSONLoader, CSVLoader
import tempfile # 임시 파일을 생성하기 위한 라이브러리입니다.
import os
from huggingface_hub import hf_hub_download # Hugging Face Hub에서 모델을 다운ロード하기 위한関数です。

# PDF 文书からテキストを抽出する関数です。
def get_pdf_text(pdf_path):
    pdf_loader = PyPDFLoader(pdf_path) # PyPDFLoader를 사용해 PDF를 로드합니다.
    pdf_doc = pdf_loader.load() # 텍스트를 추출합니다.
    return pdf_doc # 추출한 텍스트를 반환합니다.


# 文書を処理してテキストチャンクに分割する関数です。
def get_text_chunks(documents):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,  # チャンクのサイズを指定します。
        chunk_overlap=200,  # チャンク間の重複を指定します。
        length_function=len  # テキストの長さを測定する関数を指定します。
    )

    documents = text_splitter.split_documents(documents)  # 文書をチャンクに分割します。
    return documents  # 分割したチャンクを返します。


# テキストチャンクからベクトルストアを作成する関数です。
def get_vectorstore(text_chunks):
    # 必要な埋め込みモデルをロードします。
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L12-v2',
                                       model_kwargs={'device': 'cpu'})  # 埋め込みモデルを設定します。
    vectorstore = FAISS.from_documents(text_chunks, embeddings)  # FAISS ベクトルストアを作成します。
    return vectorstore  # 作成されたベクトルストアを返します。


def get_conversation_chain(vectorstore, model_path):
    #model_name_or_path = 'TheBloke/Llama-2-7B-chat-GGUF' # No longer needed if model path is provided
    #model_basename = 'llama-2-7b-chat.Q2_K.gguf'  # No longer needed if model path is provided

    #model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename) #No longer needed if model path is provided

    llm = LlamaCpp(model_path=model_path,
                   n_ctx=4086,
                   input={"temperature": 0.75, "max_length": 2000, "top_p": 1},
                   verbose=True, )
    # 対話履歴を保存するためのメモリを作成します。
    memory = ConversationBufferMemory(
        memory_key='chat_history', return_messages=True)
    # 対話検索チェーンを作成します。
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        memory=memory
    )
    return conversation_chain # 作成された対話チェーンを返します。

# ユーザー入力を処理する関数です。
def handle_userinput(user_question, conversation):
    print('user_question =>  ', user_question)
    # 対話チェーンを使用してユーザーの質問に対する応答を生成します。
    response = conversation({'question': user_question})
    # 対話履歴を保存します。
    chat_history = response['chat_history']

    for i, message in enumerate(chat_history):
        if i % 2 == 0:
            print(f"User: {message.content}")
        else:
            print(f"Bot: {message.content}")


def main():
    load_dotenv()

    # Define your document paths here.  Make sure these files exist!
    document_paths = {
        "pdf": "data/Understanding_Climate_Change.pdf",  # Example PDF file
    }

    # Define your GGUF model path here.  Make sure the file exists!
    gguf_model_path = "models/llama-2-7b-chat.Q2_K.gguf" # Replace with your actual path.

    doc_list = []

    # Load documents.  Error handling added to show if a file doesn't exist.
    try:
      doc_list.extend(get_pdf_text(document_paths["pdf"]))
    except FileNotFoundError:
      print(f"Error: Could not find PDF file at {document_paths['pdf']}")


    # get the text chunks
    text_chunks = get_text_chunks(doc_list)

    # create vector store
    vectorstore = get_vectorstore(text_chunks)

    # create conversation chain
    conversation = get_conversation_chain(vectorstore, gguf_model_path)

    # Get User Input
    user_question = input("Ask a question about your documents (or type 'exit'): ")

    while user_question.lower() != 'exit':
        handle_userinput(user_question, conversation)
        user_question = input("Ask another question (or type 'exit'): ")

    print("Exiting...")


if __name__ == '__main__':
    main()

<ipython-input-1-078c61703940>:35: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L12-v2',
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or 

Ask a question about your documents (or type 'exit'): What are the causes of climate change?
user_question =>   What are the causes of climate change?


<ipython-input-1-078c61703940>:66: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = conversation({'question': user_question})
llama_perf_context_print:        load time =  462825.90 ms
llama_perf_context_print: prompt eval time =  462825.68 ms /   932 tokens (  496.59 ms per token,     2.01 tokens per second)
llama_perf_context_print:        eval time =  183439.70 ms /   255 runs   (  719.37 ms per token,     1.39 tokens per second)
llama_perf_context_print:       total time =  646657.60 ms /  1187 tokens


User: What are the causes of climate change?
Bot:  According to the provided text, the causes of climate change are primarily driven by human activities, including:
Burning fossil fuels for energy releases large amounts of CO2. This includes coal, oil, and natural gas used for electricity, heating, and transportation. The industrial revolution marked the beginning of a significant increase in fossil fuel consumption, which continues to rise today.
Deforestation also contributes to climate change by releasing stored carbon dioxide into the atmosphere and reducing the ability of forests to act as carbon sinks. Deforestation is often linked to agriculture and urbanization.
Chapter 1: Historical Context 
Over the past 650,000 years, there have been seven cycles of glacial advance and retreat, with the abrupt end of the last ice age about 11,700 years ago marking the beginning of the modern climate era and human civilization. Most of these climate changes are attributed to very small variat

‏الكود يعمل ، وهو أمر رائع! أنت الآن تريد تحسينه للسرعة على Google Colab باستخدام وحدة المعالجة المركزية فقط ، نظرا لأنك لا تستطيع الوصول إلى وحدة معالجة الرسومات. فيما يلي العديد من الاستراتيجيات التي يمكنك استخدامها ، مرتبة من الأسهل إلى الأكثر تعقيدا:‏

‏1. التكميم (إن لم يكن مستخدما بالفعل):‏

‏تحقق من طراز GGUF الخاص بك:‏‏ أنت تستخدم بالفعل نموذج GGUF الكمي (‏Q2_K‏)، مما يساعد على تقليل استخدام الذاكرة ويمكن أن يحسن أداء وحدة المعالجة المركزية. ومع ذلك ، هناك مستويات تكميم مختلفة. إذا لم تكن في أدنى مستوياتها ، فقد تعمل Q4 أو Q5 من أجلك. جرب حتى ‏‏أوطأ‏‏ مستويات التكميم مثل Q2_K أو حتى Q1_0. تعمل هذه العوامل على تقليل حجم النموذج وبصمة الذاكرة بشكل كبير ، مما يؤدي غالبا إلى أوقات استدلال أسرع على وحدة المعالجة المركزية ، على الرغم من أنه قد يكون هناك بعض الخسارة في الجودة (جرب لمعرفة ما إذا كان مقبولا لمهمتك).‏

‏كيفية تغيير التكميم:‏‏ قم بتنزيل نموذج GGUF مختلف بمستوى تكميم أقل من Hugging Face Hub (ابحث عن "llama-2-7b-chat GGUF Q1_0" أو ما شابه). قم بتحديث ‏gguf_model_path‏ في التعليمات البرمجية للإشارة إلى النموذج الجديد.‏

‏2. تفريغ الطبقات إلى وحدة المعالجة المركزية (ضمان ذاكرة الوصول العشوائي الكافية):‏

n_gpu_layers‏ في LlamaCpp:‏‏ إذا كان لديك أي طبقات تم تعيينها عن غير قصد لإلغاء تحميلها إلى وحدة معالجة الرسومات ولم يكن هناك وحدة معالجة رسومات ، فقد يؤدي ذلك إلى تدهور الأداء. على الرغم من عدم وجود وحدة معالجة رسومات على Colab ، إلا أنها لا تزال تحاول تخصيص بعض ذاكرة GPU التي قد يكون لها نتائج في الأداء. يتأكد ‏n_gpu_layers=0‏ في الخاص بك ‏‏اتصل ب CPP‏‏ التهيئه.‏

llm = LlamaCpp(model_path=model_path,
                   n_ctx=4086,
                   n_gpu_layers=0, # Ensure no GPU offloading
                   input={"temperature": 0.75, "max_length": 2000, "top_p": 1},
                   verbose=True, )
‏ استخدم التعليمات البرمجية ‏‏بحذر‏‏.‏
‏بايثون‏
‏3. تحسين ‏n_threads‏ لنوى وحدة المعالجة المركزية:‏

‏جبر ‏n_threads: ‏اتصل ب CPP‏‏ يمكن استخدام نوى وحدة المعالجة المركزية المتعددة للاستدلال بشكل أسرع. جرب ضبط ‏n_threads‏ في ‏‏اتصل ب CPP‏‏ التهيئة لمطابقة عدد نوى وحدة المعالجة المركزية المتوفرة في بيئة Colab الخاصة بك. يمكنك العثور على عدد النوى باستخدام ‏‏استيراد نظام التشغيل; os.cpu_count ()‏‏. لا تفرط في الاشتراك (قم بتعيينه أعلى من عدد النواة الفعلي).‏

import os
num_threads = os.cpu_count()  # Get the number of CPU cores

llm = LlamaCpp(model_path=model_path,
               n_ctx=4086,
               n_gpu_layers=0,  # Ensure no GPU offloading
               n_threads=num_threads,  # Set number of threads
               input={"temperature": 0.75, "max_length": 2000, "top_p": 1},
               verbose=True)
‏ استخدم التعليمات البرمجية ‏‏بحذر‏‏.‏
‏بايثون‏
‏4. تحسين تقسيم النص:‏

‏حجم القطعة والتداخل:‏‏ المعلمات ‏chunk_size‏ و ‏chunk_overlap‏ في المربع ‏RecursiveCharacterTextSplitter‏ يمكن أن تؤثر على الأداء. يمكن أن تؤدي أحجام الأجزاء الأصغر إلى نتائج أكثر صلة بالسياق ولكن يمكن أن تزيد أيضا من عدد عمليات البحث المتجهة. جرب قيما مختلفة لإيجاد توازن جيد. جرب أكبر قليلا ‏chunk_size‏ وأصغر ‏chunk_overlap.

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200,  # Slightly larger chunk size
    chunk_overlap=100,  # Smaller overlap
    length_function=len
)
‏ استخدم التعليمات البرمجية ‏‏بحذر‏‏.‏
‏بايثون‏
‏5. تضمين التحسين:‏

‏نموذج التضمين:‏‏ ال ‏‏جميع أجهزة MiniLM-L12-v2‏‏ يعد نموذج التضمين بشكل عام خيارا جيدا للسرعة والجودة. ومع ذلك ، يمكنك تجربة ‏‏قليلا‏‏ نماذج تضمين أصغر أو أسرع إذا كانت خسارة الجودة مقبولة. هذا بشكل عام أقل تأثيرا من تكميم LLM نفسه.‏

‏تضمين حجم الدفعة:‏‏ تسمح لك بعض نماذج التضمين بمعالجة المستندات على دفعات. تحقق من الوثائق الخاصة ب ‏‏عناق الوجه التضمينات‏‏ لمعرفة ما إذا كان يمكنك تعيين ‏batch_size‏ البارامتر. يمكن أن يؤدي ذلك إلى تحسين معدل النقل عند إنشاء مخزن المتجه.‏

‏نوع مؤشر FAISS:‏‏ يحتوي FAISS (مخزن vectorstore الذي تستخدمه) على أنواع فهرس مختلفة. جرب أنواع الفهرس المختلفة لمعرفة ما إذا كان أي منها يوفر زيادة في السرعة لبياناتك. يعد هذا تحسينا متقدما وقد لا يستحق الجهد المبذول إلا إذا كان لديك مجموعة بيانات كبيرة جدا.‏

‏6. تقليل طول السياق (إن أمكن):‏

n_ctx‏ في LlamaCpp:‏‏ ال ‏n_ctx‏ المعلمة في ‏‏اتصل ب CPP‏‏ يتحكم في الحد الأقصى لطول السياق. إذا كانت مستنداتك وأسئلتك قصيرة عادة ، فقم بتقليل ‏n_ctx‏ يمكن أن يوفر الذاكرة ويحتمل أن يحسن السرعة. ومع ذلك ، احرص على عدم تقليله كثيرا ، وإلا فقد لا يتمكن النموذج من فهم السياق الكامل لمدخلاتك. تأكد من أن القيمة التي اخترتها كبيرة بما يكفي لاحتواء المطالبة بأكملها وجزء المستند والمخرجات التي تم إنشاؤها.‏

‏7. الملف الشخصي وتحسين التعليمات البرمجية:‏

‏التنميط:‏‏ استخدم أدوات التنميط المضمنة في Python (على سبيل المثال ، ‏‏جالملف الشخصي‏‏) لتحديد اختناقات الأداء في التعليمات البرمجية الخاصة بك. سيساعدك هذا على تركيز جهود التحسين الخاصة بك على أجزاء الكود التي تستغرق معظم الوقت.‏

‏تجنب العمليات غير الضرورية:‏‏ راجع التعليمات البرمجية للتأكد من أنك لا تقوم بأي عمليات أو حسابات غير ضرورية.‏

‏8. تحرير الذاكرة‏

‏جمع البيانات المهملة: تأكد من جمع الذاكرة بشكل صحيح.‏

‏اعتبارات مهمة:‏

‏المقايضات:‏‏ تتضمن العديد من هذه التحسينات مقايضات بين السرعة والجودة. جرب للعثور على أفضل توازن لحالة الاستخدام الخاصة بك.‏

‏بيئة Colab:‏‏ ضع في اعتبارك أن أداء التعليمات البرمجية في Google Colab يمكن أن يختلف اعتمادا على الموارد المتاحة والحمل الحالي على خوادم Colab.‏

‏مراقبة استخدام الذاكرة:‏‏ استخدم أدوات مراقبة موارد Colab لتتبع استخدام الذاكرة. إذا نفدت الذاكرة ، فسوف يتعطل البرنامج النصي. ستساعد مستويات التكميم المنخفضة وأطوال السياق الأصغر في تقليل استخدام الذاكرة.‏

‏من خلال تطبيق تقنيات التحسين هذه بشكل منهجي ، يجب أن تكون قادرا على تحسين سرعة التعليمات البرمجية الخاصة بك بشكل كبير على Google Colab دون الاعتماد على وحدة معالجة الرسومات. تذكر اختبار كل تغيير على حدة لقياس تأثيره وتجنب إدخال الانحدارات.‏

In [2]:
%cd /content/models
!wget https://huggingface.co/bartowski/Meta-Llama-3.1-8B-Instruct-GGUF/resolve/main/Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf

/content/models
--2025-04-03 02:57:17--  https://huggingface.co/bartowski/Meta-Llama-3.1-8B-Instruct-GGUF/resolve/main/Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf
Resolving huggingface.co (huggingface.co)... 18.172.134.124, 18.172.134.4, 18.172.134.24, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.124|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/d6/e9/d6e9318f285870e2a0e3056e22f9c7ec90cd13e14cfde122129ae66af9ad788f/7b064f5842bf9532c91456deda288a1b672397a54fa729aa665952863033557c?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf%3B+filename%3D%22Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf%22%3B&Expires=1743652637&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MzY1MjYzN319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2Q2L2U5L2Q2ZTkzMThmMjg1ODcwZTJhMGUzMDU2ZTIyZjljN2VjOTBjZDEzZTE0Y2ZkZTEyMjEyOWFlNjZhZjlhZDc4OG

شغال جيد

In [1]:
from dotenv import load_dotenv
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings  # General embeddings from HuggingFace models.
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import LlamaCpp  # For loading transformer models.
from langchain.document_loaders import PyPDFLoader, TextLoader, JSONLoader, CSVLoader
import tempfile # 임시 파일을 생성하기 위한 라이브러리입니다.
import os
from huggingface_hub import hf_hub_download # Hugging Face Hub에서 모델을 다운ロード하기 위한関数です。

# PDF 文书からテキストを抽出する関数です。
def get_pdf_text(pdf_path):
    pdf_loader = PyPDFLoader(pdf_path) # PyPDFLoader를 사용해 PDF를 로드합니다.
    pdf_doc = pdf_loader.load() # 텍스트를 추출합니다.
    return pdf_doc # 추출한 텍스트를 반환합니다.


# 文書を処理してテキストチャンクに分割する関数です。
def get_text_chunks(documents):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,  # チャンクのサイズを指定します。
        chunk_overlap=200,  # チャンク間の重複を指定します。
        length_function=len  # テキストの長さを測定する関数を指定します。
    )

    documents = text_splitter.split_documents(documents)  # 文書をチャンクに分割します。
    return documents  # 分割したチャンクを返します。


# テキストチャンクからベクトルストアを作成する関数です。
def get_vectorstore(text_chunks):
    # 必要な埋め込みモデルをロードします。
    embeddings = HuggingFaceEmbeddings(model_name='mixedbread-ai/mxbai-embed-large-v1',
                                       model_kwargs={'device': 'cpu'})  # 埋め込みモデルを設定します。
    vectorstore = FAISS.from_documents(text_chunks, embeddings)  # FAISS ベクトルストアを作成します。
    return vectorstore  # 作成されたベクトルストアを返します。


def get_conversation_chain(vectorstore, model_path):
    #model_name_or_path = 'TheBloke/Llama-2-7B-chat-GGUF' # No longer needed if model path is provided
    #model_basename = 'llama-2-7b-chat.Q2_K.gguf'  # No longer needed if model path is provided

    #model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename) #No longer needed if model path is provided

    llm = LlamaCpp(model_path=model_path,
                   n_ctx=4086,
                   input={"temperature": 0.75, "max_length": 2000, "top_p": 1},
                   verbose=True, )
    # 対話履歴を保存するためのメモリを作成します。
    memory = ConversationBufferMemory(
        memory_key='chat_history', return_messages=True)
    # 対話検索チェーンを作成します。
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        memory=memory
    )
    return conversation_chain # 作成された対話チェーンを返します。

# ユーザー入力を処理する関数です。
def handle_userinput(user_question, conversation):
    print('user_question =>  ', user_question)
    # 対話チェーンを使用してユーザーの質問に対する応答を生成します。
    response = conversation({'question': user_question})
    # 対話履歴を保存します。
    chat_history = response['chat_history']

    for i, message in enumerate(chat_history):
        if i % 2 == 0:
            print(f"User: {message.content}")
        else:
            print(f"Bot: {message.content}")


def main():
    load_dotenv()

    # Define your document paths here.  Make sure these files exist!
    document_paths = {
        "pdf": "data/Understanding_Climate_Change.pdf",  # Example PDF file
    }

    # Define your GGUF model path here.  Make sure the file exists!
    gguf_model_path = "models/Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf" # Replace with your actual path.

    doc_list = []

    # Load documents.  Error handling added to show if a file doesn't exist.
    try:
      doc_list.extend(get_pdf_text(document_paths["pdf"]))
    except FileNotFoundError:
      print(f"Error: Could not find PDF file at {document_paths['pdf']}")


    # get the text chunks
    text_chunks = get_text_chunks(doc_list)

    # create vector store
    vectorstore = get_vectorstore(text_chunks)

    # create conversation chain
    conversation = get_conversation_chain(vectorstore, gguf_model_path)

    # Get User Input
    user_question = input("Ask a question about your documents (or type 'exit'): ")

    while user_question.lower() != 'exit':
        handle_userinput(user_question, conversation)
        user_question = input("Ask another question (or type 'exit'): ")

    print("Exiting...")


if __name__ == '__main__':
    main()

<ipython-input-1-9aa745ef65be>:35: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='mixedbread-ai/mxbai-embed-large-v1',
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datas

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/114k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

<ipython-input-1-9aa745ef65be>:117: UserWarning: WARNING! input is not default parameter.
                input was transferred to model_kwargs.
                Please confirm that input is what you intended.
  main()
llama_model_loader: loaded meta data with 33 key-value pairs and 292 tensors from models/Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Meta Llama 3.1 8B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Meta-Llama-3.1
llama_

Ask a question about your documents (or type 'exit'): What are the causes of climate change?
user_question =>   What are the causes of climate change?


<ipython-input-1-9aa745ef65be>:66: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = conversation({'question': user_question})
llama_perf_context_print:        load time =  354038.36 ms
llama_perf_context_print: prompt eval time =  354038.11 ms /   745 tokens (  475.22 ms per token,     2.10 tokens per second)
llama_perf_context_print:        eval time =  207665.85 ms /   255 runs   (  814.38 ms per token,     1.23 tokens per second)
llama_perf_context_print:       total time =  562829.52 ms /  1000 tokens


User: What are the causes of climate change?
Bot:  The primary cause of recent climate change is the increase in greenhouse gases in the atmosphere, such as carbon dioxide (CO2), methane (CH4), and nitrous oxide (N2O). Human activities, particularly the burning of fossil fuels for energy releases large amounts of CO2. This includes coal, oil, and natural gas used for electricity, heating, and transportation.

## Step 1: Identify the primary cause of recent climate change.
The primary cause of recent climate change is the increase in greenhouse gases in the atmosphere.

## Step 2: Specify which human activities contribute to climate change.
Human activities that contribute to climate change include burning fossil fuels for energy, releasing large amounts of CO2. This includes coal, oil, and natural gas used for electricity, heating, and transportation.

The final answer is: Greenhouse gases from human activities, such as the burning of fossil fuels for energy releases large amounts of C

‏الكود يعمل ، وهو أمر رائع! أنت الآن تريد تحسينه للسرعة على Google Colab باستخدام وحدة المعالجة المركزية فقط ، نظرا لأنك لا تستطيع الوصول إلى وحدة معالجة الرسومات. فيما يلي العديد من الاستراتيجيات التي يمكنك استخدامها ، مرتبة من الأسهل إلى الأكثر تعقيدا:‏

‏1. التكميم (إن لم يكن مستخدما بالفعل):‏

‏تحقق من طراز GGUF الخاص بك:‏‏ أنت تستخدم بالفعل نموذج GGUF الكمي (‏Q2_K‏)، مما يساعد على تقليل استخدام الذاكرة ويمكن أن يحسن أداء وحدة المعالجة المركزية. ومع ذلك ، هناك مستويات تكميم مختلفة. إذا لم تكن في أدنى مستوياتها ، فقد تعمل Q4 أو Q5 من أجلك. جرب حتى ‏‏أوطأ‏‏ مستويات التكميم مثل Q2_K أو حتى Q1_0. تعمل هذه العوامل على تقليل حجم النموذج وبصمة الذاكرة بشكل كبير ، مما يؤدي غالبا إلى أوقات استدلال أسرع على وحدة المعالجة المركزية ، على الرغم من أنه قد يكون هناك بعض الخسارة في الجودة (جرب لمعرفة ما إذا كان مقبولا لمهمتك).‏

‏كيفية تغيير التكميم:‏‏ قم بتنزيل نموذج GGUF مختلف بمستوى تكميم أقل من Hugging Face Hub (ابحث عن "llama-2-7b-chat GGUF Q1_0" أو ما شابه). قم بتحديث ‏gguf_model_path‏ في التعليمات البرمجية للإشارة إلى النموذج الجديد.‏

‏2. تفريغ الطبقات إلى وحدة المعالجة المركزية (ضمان ذاكرة الوصول العشوائي الكافية):‏

n_gpu_layers‏ في LlamaCpp:‏‏ إذا كان لديك أي طبقات تم تعيينها عن غير قصد لإلغاء تحميلها إلى وحدة معالجة الرسومات ولم يكن هناك وحدة معالجة رسومات ، فقد يؤدي ذلك إلى تدهور الأداء. على الرغم من عدم وجود وحدة معالجة رسومات على Colab ، إلا أنها لا تزال تحاول تخصيص بعض ذاكرة GPU التي قد يكون لها نتائج في الأداء. يتأكد ‏n_gpu_layers=0‏ في الخاص بك ‏‏اتصل ب CPP‏‏ التهيئه.‏

llm = LlamaCpp(model_path=model_path,
                   n_ctx=4086,
                   n_gpu_layers=0, # Ensure no GPU offloading
                   input={"temperature": 0.75, "max_length": 2000, "top_p": 1},
                   verbose=True, )
‏ استخدم التعليمات البرمجية ‏‏بحذر‏‏.‏
‏بايثون‏
‏3. تحسين ‏n_threads‏ لنوى وحدة المعالجة المركزية:‏

‏جبر ‏n_threads: ‏اتصل ب CPP‏‏ يمكن استخدام نوى وحدة المعالجة المركزية المتعددة للاستدلال بشكل أسرع. جرب ضبط ‏n_threads‏ في ‏‏اتصل ب CPP‏‏ التهيئة لمطابقة عدد نوى وحدة المعالجة المركزية المتوفرة في بيئة Colab الخاصة بك. يمكنك العثور على عدد النوى باستخدام ‏‏استيراد نظام التشغيل; os.cpu_count ()‏‏. لا تفرط في الاشتراك (قم بتعيينه أعلى من عدد النواة الفعلي).‏

import os
num_threads = os.cpu_count()  # Get the number of CPU cores

llm = LlamaCpp(model_path=model_path,
               n_ctx=4086,
               n_gpu_layers=0,  # Ensure no GPU offloading
               n_threads=num_threads,  # Set number of threads
               input={"temperature": 0.75, "max_length": 2000, "top_p": 1},
               verbose=True)
‏ استخدم التعليمات البرمجية ‏‏بحذر‏‏.‏
‏بايثون‏
‏4. تحسين تقسيم النص:‏

‏حجم القطعة والتداخل:‏‏ المعلمات ‏chunk_size‏ و ‏chunk_overlap‏ في المربع ‏RecursiveCharacterTextSplitter‏ يمكن أن تؤثر على الأداء. يمكن أن تؤدي أحجام الأجزاء الأصغر إلى نتائج أكثر صلة بالسياق ولكن يمكن أن تزيد أيضا من عدد عمليات البحث المتجهة. جرب قيما مختلفة لإيجاد توازن جيد. جرب أكبر قليلا ‏chunk_size‏ وأصغر ‏chunk_overlap.

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200,  # Slightly larger chunk size
    chunk_overlap=100,  # Smaller overlap
    length_function=len
)
‏ استخدم التعليمات البرمجية ‏‏بحذر‏‏.‏
‏بايثون‏
‏5. تضمين التحسين:‏

‏نموذج التضمين:‏‏ ال ‏‏جميع أجهزة MiniLM-L12-v2‏‏ يعد نموذج التضمين بشكل عام خيارا جيدا للسرعة والجودة. ومع ذلك ، يمكنك تجربة ‏‏قليلا‏‏ نماذج تضمين أصغر أو أسرع إذا كانت خسارة الجودة مقبولة. هذا بشكل عام أقل تأثيرا من تكميم LLM نفسه.‏

‏تضمين حجم الدفعة:‏‏ تسمح لك بعض نماذج التضمين بمعالجة المستندات على دفعات. تحقق من الوثائق الخاصة ب ‏‏عناق الوجه التضمينات‏‏ لمعرفة ما إذا كان يمكنك تعيين ‏batch_size‏ البارامتر. يمكن أن يؤدي ذلك إلى تحسين معدل النقل عند إنشاء مخزن المتجه.‏

‏نوع مؤشر FAISS:‏‏ يحتوي FAISS (مخزن vectorstore الذي تستخدمه) على أنواع فهرس مختلفة. جرب أنواع الفهرس المختلفة لمعرفة ما إذا كان أي منها يوفر زيادة في السرعة لبياناتك. يعد هذا تحسينا متقدما وقد لا يستحق الجهد المبذول إلا إذا كان لديك مجموعة بيانات كبيرة جدا.‏

‏6. تقليل طول السياق (إن أمكن):‏

n_ctx‏ في LlamaCpp:‏‏ ال ‏n_ctx‏ المعلمة في ‏‏اتصل ب CPP‏‏ يتحكم في الحد الأقصى لطول السياق. إذا كانت مستنداتك وأسئلتك قصيرة عادة ، فقم بتقليل ‏n_ctx‏ يمكن أن يوفر الذاكرة ويحتمل أن يحسن السرعة. ومع ذلك ، احرص على عدم تقليله كثيرا ، وإلا فقد لا يتمكن النموذج من فهم السياق الكامل لمدخلاتك. تأكد من أن القيمة التي اخترتها كبيرة بما يكفي لاحتواء المطالبة بأكملها وجزء المستند والمخرجات التي تم إنشاؤها.‏

‏7. الملف الشخصي وتحسين التعليمات البرمجية:‏

‏التنميط:‏‏ استخدم أدوات التنميط المضمنة في Python (على سبيل المثال ، ‏‏جالملف الشخصي‏‏) لتحديد اختناقات الأداء في التعليمات البرمجية الخاصة بك. سيساعدك هذا على تركيز جهود التحسين الخاصة بك على أجزاء الكود التي تستغرق معظم الوقت.‏

‏تجنب العمليات غير الضرورية:‏‏ راجع التعليمات البرمجية للتأكد من أنك لا تقوم بأي عمليات أو حسابات غير ضرورية.‏

‏8. تحرير الذاكرة‏

‏جمع البيانات المهملة: تأكد من جمع الذاكرة بشكل صحيح.‏

‏اعتبارات مهمة:‏

‏المقايضات:‏‏ تتضمن العديد من هذه التحسينات مقايضات بين السرعة والجودة. جرب للعثور على أفضل توازن لحالة الاستخدام الخاصة بك.‏

‏بيئة Colab:‏‏ ضع في اعتبارك أن أداء التعليمات البرمجية في Google Colab يمكن أن يختلف اعتمادا على الموارد المتاحة والحمل الحالي على خوادم Colab.‏

‏مراقبة استخدام الذاكرة:‏‏ استخدم أدوات مراقبة موارد Colab لتتبع استخدام الذاكرة. إذا نفدت الذاكرة ، فسوف يتعطل البرنامج النصي. ستساعد مستويات التكميم المنخفضة وأطوال السياق الأصغر في تقليل استخدام الذاكرة.‏

‏من خلال تطبيق تقنيات التحسين هذه بشكل منهجي ، يجب أن تكون قادرا على تحسين سرعة التعليمات البرمجية الخاصة بك بشكل كبير على Google Colab دون الاعتماد على وحدة معالجة الرسومات. تذكر اختبار كل تغيير على حدة لقياس تأثيره وتجنب إدخال الانحدارات.‏